In [1]:
import pandas as pd
import numpy as np

In [2]:
q = '''
SELECT
  DISTINCT practice,
  pct
FROM
  ebmdatalab.hscic.normalised_prescribing_standard
LEFT JOIN
  ebmdatalab.hscic.practices
ON
  practice = code
  AND setting = 4
ORDER BY
  practice
'''
practices = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')


phe = pd.read_csv('indicators-GP.data.csv')

mask = (phe['Indicator Name'] == 'Deprivation score (IMD 2015)') & (phe['Area Type'] == 'GP')
imd = phe.loc[mask, ['Area Code','Value']]
imd = imd.set_index('Area Code')

mask = (phe['Indicator Name'] == '% who would recommend practice') & (phe['Area Type'] == 'GP') & (phe['Time period'] == '2015/16')
satisfaction = phe.loc[mask, ['Area Code','Value']]
satisfaction = satisfaction.set_index('Area Code')

mask = (phe['Indicator Name'] == '% with a long-standing health condition') & (phe['Area Type'] == 'GP') & (phe['Time period'] == '2015/16')
long_term_health = phe.loc[mask, ['Area Code','Value']]
long_term_health = long_term_health.set_index('Area Code')

mask = (phe['Indicator Name'] == '% aged 65+ years') & (phe['Area Type'] == 'GP') & (phe['Time period'] == 2016)
over_65 = phe.loc[mask, ['Area Code','Value']]
over_65 = over_65.set_index('Area Code')

mask = (phe['Indicator Name'] == '% aged under 18 years') & (phe['Area Type'] == 'GP') & (phe['Time period'] == 2016)
under_18 = phe.loc[mask, ['Area Code','Value']]
under_18 = under_18.set_index('Area Code')



qof = pd.read_csv('ACHIEVEMENT_EXCEPTIONS.csv')
qof_map = pd.read_csv('INDICATOR_MAPPINGS.csv')
qof = qof.merge(qof_map[['INDICATOR_CODE','DOMAIN_CODE']], how='inner', on='INDICATOR_CODE', copy=False)
qof_all = qof.loc[qof.MEASURE == 'ACHIEVED_POINTS'].groupby(['PRACTICE_CODE','DOMAIN_CODE']).sum()
qof_all = qof_all.unstack(level=1) #.reset_index(col_level=0)
qof_all.columns = qof_all.columns.droplevel(0)
qof_all['TOTAL'] = qof_all['CL'] + qof_all['PH'] + qof_all['PHAS']



q = '''
SELECT
  code,
  dispensing_patients
FROM
  ebmdatalab.bsa.dispensing_practices_jan2017
ORDER BY
  code ASC
'''
dispensing = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')



q = '''
SELECT
  practice,
  SUM(items) AS total_items
FROM
  ebmdatalab.hscic.normalised_prescribing_standard
WHERE
  month >= TIMESTAMP("2017-01-01")
  AND month <= TIMESTAMP("2017-12-01")
GROUP BY
  practice
'''
prescribing_volume = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')



q = '''
SELECT
  *
FROM
  ebmdatalab.ONS.practices_rural_urban
'''
urban_rural = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')


q = '''
SELECT
  practice,
  AVG(total_list_size) as list_size
FROM
  ebmdatalab.hscic.practice_statistics
WHERE
  month >= TIMESTAMP("2017-01-01")
  AND month <= TIMESTAMP("2017-12-01")
GROUP BY
  practice
ORDER BY
  practice
'''
list_size = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')



gps_per_practice = pd.read_csv('gps_per_practice_sept_2017.csv')



measures = {'ktt9_cephalosporins':'Antibiotic stewardship: co-amoxiclav, cephalosporins &amp; quinolones (KTT9)',
            'ktt9_uti_antibiotics':'Antibiotic stewardship: three-day courses for uncomplicated UTIs (KTT9)',
            'ktt9_antibiotics':'Antibiotic stewardship: volume of antibiotic prescribing (KTT9)',
            'ciclosporin':'Ciclosporin and tacrolimus oral preparations prescribed generically',
            'coproxamol':'Co-proxamol',
            'desogestrel':'Desogestrel prescribed as a branded product',
            'diltiazem':'Diltiazem preparations (>60mg) prescribed generically',
            #'doacs':'Direct Oral Anticoagulants (DOACs)',
            'quetiapine':'Extended-release quetiapine',
            'glaucoma':'Glaucoma eye drops prescribed by brand',
            'ace':'High-cost ACE inhibitors',
            'arb':'High-cost ARBs',
            'sildenafil':'High-cost drugs for erectile dysfunction',
            'ppi':'High-cost PPIs',
            'statins':'High-cost statins',
            'icsdose':'High dose inhaled corticosteroids',
            'opioidspercent':'High dose opioids as percentage regular opioids',
            'opioidper1000':'High dose opioids per 1000 patients',
            'ppidose':'Higher dose Proton Pump Inhibitors (PPIs)',
            'keppra':'Keppra vs. levetiracetam',
            'ktt12_diabetes_insulin':'Long-acting insulin analogues (KTT12)',
            'statinintensity':'Low and medium intensity statins',
            'methotrexate':'Methotrexate 10 mg tablets',
            'nebivolol':'Nebivolol 2.5mg tablets',
            'lpzomnibus':'NHS England Low Priority Treatment - All Low Priority Treatments',
            #'lpcoprox':'NHS England Low Priority Treatment - co-proxamol',
            #'lpdosulepin':'NHS England Low Priority Treatment - dosulepin',
            #'lpdoxazosin':'NHS England Low Priority Treatment - doxazosin modified release',
            #'lpfentanylir':'NHS England Low Priority Treatment - fentanyl immediate release',
            #'lpglucosamine':'NHS England Low Priority Treatment - glucosamine and chondroitin',
            #'lphomeopathy':'NHS England Low Priority Treatment - homeopathy',
            #'lplidocaine':'NHS England Low Priority Treatment - lidocaine plasters',
            #'lpliothyronine':'NHS England Low Priority Treatment - liothyronine',
            #'lplutein':'NHS England Low Priority Treatment - lutein and antioxidants',
            #'lpomega3':'NHS England Low Priority Treatment - omega-3 fatty acid compounds',
            #'lpoxycodone':'NHS England Low Priority Treatment - oxycodone and naloxone combination product',
            #'lptramadolpara':'NHS England Low Priority Treatment - paracetamol and tramadol combination',
            #'lpperindopril':'NHS England Low Priority Treatment - perindopril arginine',
            #'lprubefacients':'NHS England Low Priority Treatment - rubefacients',
            #'lptadalafil':'NHS England Low Priority Treatment - tadalafil once daily',
            #'lptravelvacs':'NHS England Low Priority Treatment - travel vaccines',
            #'lptrimipramine':'NHS England Low Priority Treatment - trimipramine',
            #'glutenfree':'Prescribing of gluten free products',
            'ktt13_nsaids_ibuprofen':'Non-preferred NSAIDs and COX-2 inhibitors (KTT13)',
            'other_lipid_modifying_drugs':'Other lipid-modifying drugs',
            'lyrica':'Pregabalin prescribed as Lyrica',
            'dipyridamole':'Prescribing of dipyridamole',
            'tramadol':'Prescribing of high cost tramadol preparations ',
            #'pregabalin':'Prescribing of pregabalin',
            #'pregabalinmg':'Prescribing of pregabalin (total mg)',
            'trimethoprim':'Prescribing of trimethoprim vs nitrofurantoin',
            'saba':'Short acting beta agonist inhalers',
            'silver':'Silver dressings',
            'solublepara':'Soluble/effervescent forms of paracetamol and co-codamol',
            'fungal':'Topical treatment of fungal nail infections',
            'vitb':'Vitamin B complex',
            'bdzper1000':'Anxiolytics and Hypnotics: Average Daily Quantity per 1000 patients',
            'bdzadq':'Anxiolytics and Hypnotics: Average Daily Quantity per item',
            'opioidome':'Prescribing of opioids (total oral morphine equivalence)'
}
all_measures = []
for measure, label in measures.items():
    q = '''
    SELECT
      practice_id,
      AVG(percentile) as mean_percentile
    FROM
      ebmdatalab.measures.practice_data_%s
    WHERE
      month >= "2017-01-01"
      AND month <= "2017-12-01"
    GROUP BY
      practice_id
    ''' % (measure)

    df = pd.read_gbq(q, 'ebmdatalab', verbose=False, dialect='standard')
    df['measure'] = measure
    all_measures.append(df)
    del df
all_measures = pd.concat(all_measures)
summ_measures = all_measures.groupby('practice_id').mean()


software = pd.read_csv("GPSoC Marketshare - Jan 2018b.csv", usecols = ['ODS','Principal Supplier','Principal System'])

C:\Users\ajwalker\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9,10,11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
all = practices.merge(urban_rural,
                how='left',
                left_on='practice',
                right_on='code',
                copy=False)

all = all.merge(summ_measures,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False)

all = all.merge(gps_per_practice,
                how='left',
                on='practice',
                copy=False)

all = all.merge(qof_all,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False)

all = all.merge(imd,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False)
all = all.merge(satisfaction,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False,
                suffixes=('_imd', ''))
all = all.merge(long_term_health,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False,
                suffixes=('_satisfaction', ''))
all = all.merge(over_65,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False,
                suffixes=('_long_term_health', ''))
all = all.merge(under_18,
                how='left',
                left_on='practice',
                right_index=True,
                copy=False,
                suffixes=('_over_65', '_under_18'))
all = all.merge(dispensing,
                how='left',
                left_on='practice',
                right_on='code',
                copy=False)
all = all.merge(prescribing_volume,
                how='left',
                left_on='practice',
                right_on='practice',
                copy=False)
all = all.merge(list_size,
                how='left',
                left_on='practice',
                right_on='practice',
                copy=False)
all = all.merge(software,
                how='left',
                left_on='practice',
                right_on='ODS',
                copy=False)

#all.to_csv('methotrexate_for_analysis.csv')
all.describe()
all = all.drop(columns=['code_x','code_y','ODS'])
all.head()

,practice,pct,rural_urban_code,mean_percentile,num_gps_june,num_gps_sept,CL,PH,PHAS,TOTAL,Value_imd,Value_satisfaction,Value_long_term_health,Value_over_65,Value_under_18,dispensing_patients,total_items,list_size,Principal Supplier,Principal System
0,A81001,00K,4,0.404129,2.0,NaN,425.69,97.00,24.0,546.69,29.140310,76.020488,57.895013,22.674829,19.722157,0.0,92445.0,4160.416667,TPP,SystmOne
1,A81002,00K,4,0.495510,12.0,12.0,435.00,97.00,27.0,559.00,29.689706,85.166335,58.872464,21.025589,18.776949,0.0,457691.0,19945.500000,TPP,SystmOne
2,A81003,00K,4,NaN,NaN,NaN,425.34,97.00,27.0,549.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A81004,00M,4,0.492901,2.0,3.0,427.63,97.00,27.0,551.63,34.411828,72.955168,50.053075,18.898240,18.135315,0.0,197867.0,9572.250000,TPP,SystmOne
4,A81005,00M,3,0.374380,8.0,9.0,429.72,91.76,27.0,548.48,15.743812,85.118831,52.133158,26.626843,17.585155,0.0,193415.0,7957.583333,TPP,SystmOne


In [4]:
all

,practice,pct,rural_urban_code,mean_percentile,num_gps_june,num_gps_sept,CL,PH,PHAS,TOTAL,Value_imd,Value_satisfaction,Value_long_term_health,Value_over_65,Value_under_18,dispensing_patients,total_items,list_size,Principal Supplier,Principal System
0,A81001,00K,4,0.404129,2.0,NaN,425.69,97.00,24.00,546.69,29.140310,76.020488,57.895013,22.674829,19.722157,0.0,92445.0,4160.416667,TPP,SystmOne
1,A81002,00K,4,0.495510,12.0,12.0,435.00,97.00,27.00,559.00,29.689706,85.166335,58.872464,21.025589,18.776949,0.0,457691.0,19945.500000,TPP,SystmOne
2,A81003,00K,4,NaN,NaN,NaN,425.34,97.00,27.00,549.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A81004,00M,4,0.492901,2.0,3.0,427.63,97.00,27.00,551.63,34.411828,72.955168,50.053075,18.898240,18.135315,0.0,197867.0,9572.250000,TPP,SystmOne
4,A81005,00M,3,0.374380,8.0,9.0,429.72,91.76,27.00,548.48,15.743812,85.118831,52.133158,26.626843,17.585155,0.0,193415.0,7957.583333,TPP,SystmOne
5,A81006,00K,4,0.426344,8.0,8.0,425.72,97.00,27.00,549.72,31.791515,85.096769,51.838239,18.484408,20.861205,0.0,330733.0,13844.916667,TPP,SystmOne
6,A81007,00K,4,0.405379,9.0,10.0,430.68,97.00,27.00,554.68,32.068038,86.194289,58.507732,18.595632,21.475811,0.0,195468.0,9959.250000,EMIS,EMIS Web
7,A81008,00M,3,0.000000,NaN,NaN,428.28,97.00,27.00,552.28,NaN,NaN,NaN,NaN,NaN,NaN,32.0,3973.000000,NaN,NaN
8,A81009,00M,4,0.372182,5.0,5.0,434.00,87.00,27.00,548.00,35.263878,66.261000,60.345235,15.063402,22.114123,0.0,210899.0,8956.833333,TPP,SystmOne
9,A81011,00K,4,0.395113,5.0,5.0,434.96,93.21,27.00,555.17,34.627062,88.087260,58.803783,18.794049,19.686766,0.0,269270.0,11789.416667,TPP,SystmOne


In [5]:
all.to_csv('data_for_regression_2018_10.csv', index=False)